#### Imports

In [1]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
import seaborn as sns
from opendatasets import download_kaggle_dataset
from random import randint
from collections import defaultdict
import ipaddress as ip
import re
from sklearn.preprocessing import LabelEncoder

In [2]:
# !pip install opendatasets

# Machine learning course project

## Predict if the Merchant is Fraudster

### Author: Georgi Datskov
Softuni username: Datskov

E-mail: g.datskov@gmail.com

### Short problem description: Predict if the Merchant is Fraudster or not for an e-commerce client
#### Dataset owner problem description:
‘XYZ’ is a large e-commerce company with its operations in several countries. As the online giant grows, so has
the number of fraudster merchants are. They deliver counterfeits or, in some cases, nothing at all. Such
schemes leave customers duped, and place both legitimate merchants and the company itself in a constant
battle to rid the marketplace of scammers. Determining this is also important in budgeting for fraud
investigation. It's a well-known problem both to the company and to merchants, which they say hasn't
effectively addressed the issue. They are serious about it and want to protect themselves from these fraudulent
merchants using technology.
You are expected to create an analytical and modelling framework to predict the Merchant Fraudulency(yes/no)
based on the quantitative and qualitative features provided in the dataset while answering other questions too
cited below.

II.The datasets are provided as cited below:

Target attribute: "fraudster" (yes – 1, no – 0)

Train:

• trainmerchantdata.csv : Merchant Information

• trainorderdata.csv : Order Information

• train.csv : Target Label Information

Test:

• testmerchantdata.csv : Merchant Information

• testorderdata.csv : Order Information

• test.csv : Target is not available as it is to be predicted

• ipboundariescountries.csv : IP addresses boundaries for each country

(common for both train and test)

All Attributes names are self-explanatory.

III.Tasks:

Model Building:

You are expected to create an analytical and modelling framework to predict the Merchant Fraudulency based
on the quantitative and qualitative features provided in the datasets. You may derive new features from the
existing features and also from the domain knowledge, which may help in improving the model efficiency.

Visualization Tasks:
Exploratory Data Analysis using visualizations in R Notebook or Jupiter notebook format. (all train data to be
used for this task)

• List down the insights/patterns observed from the visualizations

• Explain the impact of most important attributes on target attribute observed from the
visualizations.

Observations:

Is there any overfitting or underfitting problem? If yes, how do you address it?

IV. Evaluation Metric:

• Consider ‘F1-score’ of the fraudulent class as the error metric for classification task to tune the

model and for submissions in the tool.

V. Hints

Both Python and R provides functions to convert IP string to numeric format which makes the number
comparison easier.

#### Dataset and problem source: Kaggle
https://www.kaggle.com/datasets/girishvutukuri/predict-if-the-merchant-is-fraudster

## Dataset EDA

### Loading the main training datasets and looking around

In [194]:
download_kaggle_dataset('https://www.kaggle.com/datasets/girishvutukuri/predict-if-the-merchant-is-fraudster', './data')

Skipping, found downloaded files in "./data\predict-if-the-merchant-is-fraudster" (use force=True to force download)


In [195]:
merchant_dataset_train = pd.read_csv('data/predict-if-the-merchant-is-fraudster/train_merchant_data-1561627820784.csv')
merchant_dataset_train

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172
...,...,...,...,...,...,...,...
54208,1746213,222372,2018-05-01 21:58:19,KGVMENGCIAVOK,M,43,247.25.244.31
54209,1746213,73491,2018-06-11 19:54:13,QJGPOYDONTYCN,M,41,80.30.170.122
54210,1746213,176456,2018-01-06 20:05:55,ZOJNAJMGNRQPZ,F,36,188.39.154.217
54211,1746213,228277,2018-03-09 07:08:10,MQAOMNZFAHZLS,M,18,116.20.101.178


In [196]:
order_dataset_train = pd.read_csv('data/predict-if-the-merchant-is-fraudster/train_order_data-1561627847149.csv')
order_dataset_train

,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID
0,126221,37cea9512f8d,2018-04-29 16:39:26,148,Direct,Credit Card,124231
1,115471,09f12e6efde2,2018-06-16 17:05:40,145,SEO,Credit Card,136178
2,151786,4e69e956e159,2018-10-26 18:00:46,62,Ads,Internet Banking,198611
3,140456,663443aaeb82,2018-12-12 05:41:52,28,SEO,Debit Card,127993
4,114721,99258810c121,2018-09-20 11:06:10,70,Ads,Credit Card,250146
...,...,...,...,...,...,...,...
54208,143943,4b0857877884,2018-06-01 14:39:47,90,Ads,Credit Card,61951
54209,125429,2de2749f5442,2018-08-05 21:58:02,120,SEO,Debit Card,328145
54210,159393,11635fac40ff,2018-06-23 01:54:24,92,Ads,Credit Card,28307
54211,133631,6bbcff2f14e5,2018-06-15 21:54:36,148,Ads,Cash On Delivery,277466


In [197]:
order_dataset_test = pd.read_csv('data/predict-if-the-merchant-is-fraudster/test_order_data-1561627931868.csv')
order_dataset_test

,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID
0,157068,a032de091f51,2018-03-09 09:09:42,60,SEO,Credit Card,53637
1,112534,0d563f0606d6,2018-06-13 20:11:19,28,SEO,Credit Card,243517
2,148774,18fb0fa888b6,2018-04-14 04:19:36,75,Ads,Debit Card,343640
3,114528,ed1eb920d721,2018-04-01 13:55:44,98,SEO,Credit Card,69889
4,120940,313dcf962627,2018-05-25 02:08:18,35,SEO,Credit Card,125706
...,...,...,...,...,...,...,...
13549,153457,f0caea3c0bbb,2018-06-25 23:00:41,92,SEO,Credit Card,279884
13550,139271,a23964a17f12,2018-07-14 19:20:44,128,SEO,Internet Banking,206253
13551,156228,28e0b6aa4aca,2018-03-05 12:08:52,98,Direct,E-wallet,313049
13552,160078,3e1d365937d6,2018-05-22 18:30:43,68,SEO,Debit Card,304081


### Merchant Dataset

#### IP_Adress and corresponding country

In [198]:
ip_adress_country = pd.read_csv('data/predict-if-the-merchant-is-fraudster/ip_boundaries_countries-1561628631121.csv')
ip_adress_country

,lower_bound_ip_address,upper_bound_ip_address,country
0,1.0.0.0,1.0.0.255,Australia
1,1.0.1.0,1.0.1.255,China
2,1.0.2.0,1.0.3.255,China
3,1.0.4.0,1.0.7.255,Australia
4,1.0.8.0,1.0.15.255,China
...,...,...,...
138841,223.255.240.0,223.255.243.255,Hong Kong
138842,223.255.244.0,223.255.247.255,India
138843,223.255.252.0,223.255.253.255,China
138844,223.255.254.0,223.255.254.255,Singapore


In [199]:
ip_adress_country.loc[1]['lower_bound_ip_address']

'1.0.1.0'

In [200]:
target_dataset_train = pd.read_csv('data/predict-if-the-merchant-is-fraudster/train-1561627878332.csv')
target_dataset_train

,Merchant_ID,Fraudster
0,221592,0
1,316935,1
2,38454,1
3,214437,1
4,296240,1
...,...,...
54208,243300,0
54209,219957,0
54210,101017,0
54211,50557,0


In [201]:
len(order_dataset_train['Merchant_ID'].unique())

54213

In [202]:
merchant_all_data_train = pd.merge(left=merchant_dataset_train, right=target_dataset_train, on='Merchant_ID')
merchant_all_data_train

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,0
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,0
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,0
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,0
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,0
...,...,...,...,...,...,...,...,...
54208,1746213,222372,2018-05-01 21:58:19,KGVMENGCIAVOK,M,43,247.25.244.31,0
54209,1746213,73491,2018-06-11 19:54:13,QJGPOYDONTYCN,M,41,80.30.170.122,1
54210,1746213,176456,2018-01-06 20:05:55,ZOJNAJMGNRQPZ,F,36,188.39.154.217,0
54211,1746213,228277,2018-03-09 07:08:10,MQAOMNZFAHZLS,M,18,116.20.101.178,0


Now, let's add the country corresponding to each ip adress. That's gonna be quite painful, but let's try.

In [203]:
ip_adress_country.loc[1]

lower_bound_ip_address      1.0.1.0
upper_bound_ip_address    1.0.1.255
country                       China
Name: 1, dtype: object

In [204]:
ip_adress_country.loc[::10]

,lower_bound_ip_address,upper_bound_ip_address,country
0,1.0.0.0,1.0.0.255,Australia
10,1.1.1.0,1.1.1.255,Australia
20,1.2.3.0,1.2.3.255,Australia
30,1.2.64.0,1.2.127.255,China
40,1.4.16.0,1.4.31.255,China
...,...,...,...
138800,223.168.0.0,223.175.255.255,Korea Republic of
138810,223.204.0.0,223.207.255.255,Thailand
138820,223.223.164.0,223.223.167.255,Japan
138830,223.248.0.0,223.251.255.255,China


In [205]:
ip_adress_country[ip_adress_country['country'] == 'China']

,lower_bound_ip_address,upper_bound_ip_address,country
1,1.0.1.0,1.0.1.255,China
2,1.0.2.0,1.0.3.255,China
4,1.0.8.0,1.0.15.255,China
6,1.0.32.0,1.0.63.255,China
9,1.1.0.0,1.1.0.255,China
...,...,...,...
138832,223.252.128.0,223.252.255.255,China
138834,223.254.0.0,223.254.255.255,China
138835,223.255.0.0,223.255.127.255,China
138840,223.255.236.0,223.255.239.255,China


Default dict - not a good idea. Nested dictionary file (json) gets gigabytes of code just after a few thousand entries (140k * few more thousand in ranges creates gazillions of nested structure codelines)

In [206]:
'223.255.236.0' < '223.255.239.255'

True

But not always...

In [207]:
ip_adress_country[(ip_adress_country['lower_bound_ip_address'] > '247.25.244.31')]

,lower_bound_ip_address,upper_bound_ip_address,country
145,3.0.0.0,3.255.255.255,United States
146,4.0.0.0,4.255.255.255,United States
147,5.0.0.0,5.0.255.255,Syrian Arab Republic
148,5.1.0.0,5.1.31.255,Ukraine
149,5.1.32.0,5.1.39.255,Spain
...,...,...,...
28171,99.198.96.0,99.198.127.255,United States
28172,99.199.0.0,99.199.255.255,Canada
28173,99.200.0.0,99.207.255.255,United States
28174,99.208.0.0,99.223.255.255,Canada


In [208]:
'3.0.0.0' < '247.25.244.31'

False

In [209]:
asd = ip.IPv4Address('247.25.244.31')

In [210]:
print(int(asd))

4145673247


Basically, direct string to int conversion wont work...

But, IPv4 to decimal converter works:

In [211]:
merchant_dataset_train['IP_converted_to_int'] = merchant_dataset_train['IP_Address'].apply(lambda x: int(ip.IPv4Address(x)))

In [212]:
ip_adress_country_int_converted = ip_adress_country.copy()
ip_adress_country_int_converted['lower_bound_ip_address'] = ip_adress_country_int_converted['lower_bound_ip_address'].apply(lambda x: int(ip.IPv4Address(x)))
ip_adress_country_int_converted['upper_bound_ip_address'] = ip_adress_country_int_converted['upper_bound_ip_address'].apply(lambda x: int(ip.IPv4Address(x)))

In [213]:
ip_adress_country_int_converted

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China
...,...,...,...
138841,3758092288,3758093311,Hong Kong
138842,3758093312,3758094335,India
138843,3758095360,3758095871,China
138844,3758095872,3758096127,Singapore


In [214]:
ip_adress_country

,lower_bound_ip_address,upper_bound_ip_address,country
0,1.0.0.0,1.0.0.255,Australia
1,1.0.1.0,1.0.1.255,China
2,1.0.2.0,1.0.3.255,China
3,1.0.4.0,1.0.7.255,Australia
4,1.0.8.0,1.0.15.255,China
...,...,...,...
138841,223.255.240.0,223.255.243.255,Hong Kong
138842,223.255.244.0,223.255.247.255,India
138843,223.255.252.0,223.255.253.255,China
138844,223.255.254.0,223.255.254.255,Singapore


After converting the IP's from the dataset and the IP-to-Country table, we can just compare if each datapoint (by grouping them) is within each interval, corresponding to a country. And step by step adding countries for each interval group.

WARNING! Processing the IP-COUNTRY data will take some time (5-10 minutes at least!)

##### EDA PIPE:  !!! COUNTRY EXTRACTOR TURNED OFF

##### TODO: ADD TO PIPE EDA

In [288]:
merchant_dataset_train['Country'] = np.nan

##### TODO: ADD TO PIPE EDA

In [217]:
merchant_dataset_train['Country'].value_counts(dropna=False)

NaN    54213
Name: Country, dtype: int64

In [218]:
7811/54213

0.14407983325032742

Seems there is some missing data :(

In [219]:
merchant_dataset_train['Country'].value_counts(dropna=False)

NaN    54213
Name: Country, dtype: int64

In [220]:
merchant_dataset_train[merchant_dataset_train['Country'].isnull()]

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,IP_converted_to_int,Country
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,815237196,NaN
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,1577685417,NaN
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,979279225,NaN
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,3250268602,NaN
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,2103448748,NaN
...,...,...,...,...,...,...,...,...,...
54208,1746213,222372,2018-05-01 21:58:19,KGVMENGCIAVOK,M,43,247.25.244.31,4145673247,NaN
54209,1746213,73491,2018-06-11 19:54:13,QJGPOYDONTYCN,M,41,80.30.170.122,1344187002,NaN
54210,1746213,176456,2018-01-06 20:05:55,ZOJNAJMGNRQPZ,F,36,188.39.154.217,3156712153,NaN
54211,1746213,228277,2018-03-09 07:08:10,MQAOMNZFAHZLS,M,18,116.20.101.178,1947493810,NaN


In [221]:
ip_adress_country_int_converted[ip_adress_country_int_converted['lower_bound_ip_address'] <= 175607904]

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China
...,...,...,...
910,100646912,100663295,Russian Federation
911,100663296,117440511,United States
912,117440512,134217727,United States
913,134217728,150994943,United States


In [222]:
ip_adress_country['lower_bound_ip_address'][ip_adress_country['lower_bound_ip_address'].str.startswith('239')]

Series([], Name: lower_bound_ip_address, dtype: object)

Really, there is no way of telling how to fill-up the gaps of missing info. Label them as "Unknown", perhaps? Dropping 15% of the data maybe?

#### Registered_Device_ID

If we logically assume that merchant id's and registered device id's must be unique for each merchant, let's check if this is true:

In [223]:
device_id_value_counts = merchant_all_data_train['Registered_Device_ID'].value_counts()
device_id_value_counts

RJEAMRBBMPWEA    9
VGGTRBYDZMCPN    9
TCFHYPEGJMBIZ    9
RDLIARTBSYNJX    9
AZPFJFPBMDWHC    9
                ..
UFDAFHLTCAQKS    1
MQOBOMWKGTMXP    1
RTGMZDCFPGCGY    1
ZTJBKRQRGPKSW    1
MQAOMNZFAHZLS    1
Name: Registered_Device_ID, Length: 51291, dtype: int64

We can clearly see that a lot of merchants have a registered device with the same ID, which seems quite suspicious.

Lets take one of these to see which merchants use 'the same' device:

In [224]:
merchant_all_data_train[merchant_all_data_train['Registered_Device_ID'] == 'RJEAMRBBMPWEA']

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
5627,1746213,108877,2018-01-08 21:29:18,RJEAMRBBMPWEA,F,37,66.198.113.69,1
9986,1746213,99979,2018-01-08 21:29:08,RJEAMRBBMPWEA,F,37,66.198.113.69,0
17298,1746213,192190,2018-01-08 21:29:10,RJEAMRBBMPWEA,F,37,66.198.113.69,1
22205,1746213,144263,2018-01-08 21:29:11,RJEAMRBBMPWEA,F,37,66.198.113.69,1
24782,1746213,683,2018-01-08 21:29:13,RJEAMRBBMPWEA,F,37,66.198.113.69,1
25303,1746213,276773,2018-01-08 21:29:14,RJEAMRBBMPWEA,F,37,66.198.113.69,1
44412,1746213,387903,2018-01-08 21:29:21,RJEAMRBBMPWEA,F,37,66.198.113.69,1
47707,1746213,166549,2018-01-08 21:29:19,RJEAMRBBMPWEA,F,37,66.198.113.69,1
53118,1746213,245707,2018-01-08 21:29:20,RJEAMRBBMPWEA,F,37,66.198.113.69,1


Yeah. 
We can clearly see that there are multiple fraudster (maybe?) merchants which are using the same device and one who is 'legit' (maybe?). We could call that one 'The Victim' for short.

Lets check a few more:

#### Some fraudster detection just from EDA

In [225]:
def get_samples_from_counts(value_counts_serie, df_to_extract_from, df_column, number_of_random_samples):
    # A function to get samples from a pd.Dataframe based on a df['column'].value_counts()
    # Parameters are self-explanatory
    max_num_counts = max(value_counts_serie)
    sample_selection_list = [randint(2, max_num_counts) for _ in range(number_of_random_samples)]
    random_counts_selection = [value_counts_serie[value_counts_serie == x].index[randint(0, len(value_counts_serie[value_counts_serie == x])-1)] for x in sample_selection_list]
    for x in random_counts_selection:
        display(df_to_extract_from[df_to_extract_from[df_column] == x])

In [226]:
get_samples_from_counts(device_id_value_counts, merchant_all_data_train, 'Registered_Device_ID', 5)

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
5627,1746213,108877,2018-01-08 21:29:18,RJEAMRBBMPWEA,F,37,66.198.113.69,1
9986,1746213,99979,2018-01-08 21:29:08,RJEAMRBBMPWEA,F,37,66.198.113.69,0
17298,1746213,192190,2018-01-08 21:29:10,RJEAMRBBMPWEA,F,37,66.198.113.69,1
22205,1746213,144263,2018-01-08 21:29:11,RJEAMRBBMPWEA,F,37,66.198.113.69,1
24782,1746213,683,2018-01-08 21:29:13,RJEAMRBBMPWEA,F,37,66.198.113.69,1
25303,1746213,276773,2018-01-08 21:29:14,RJEAMRBBMPWEA,F,37,66.198.113.69,1
44412,1746213,387903,2018-01-08 21:29:21,RJEAMRBBMPWEA,F,37,66.198.113.69,1
47707,1746213,166549,2018-01-08 21:29:19,RJEAMRBBMPWEA,F,37,66.198.113.69,1
53118,1746213,245707,2018-01-08 21:29:20,RJEAMRBBMPWEA,F,37,66.198.113.69,1


,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
7058,1746213,42894,2018-01-07 09:06:57,GILFHMYGVYDDA,M,34,182.218.210.186,1
25130,1746213,124391,2018-01-07 09:06:52,GILFHMYGVYDDA,M,34,182.218.210.186,1
38295,1746213,222810,2018-01-07 09:06:54,GILFHMYGVYDDA,M,34,182.218.210.186,1


,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
5199,1746213,36678,2018-01-05 22:40:20,VHBEGHEWBEZNJ,F,34,195.162.9.84,1
6032,1746213,392659,2018-01-05 22:40:15,VHBEGHEWBEZNJ,F,34,195.162.9.84,1
11985,1746213,188055,2018-01-05 22:40:19,VHBEGHEWBEZNJ,F,34,195.162.9.84,1
21664,1746213,293689,2018-01-05 22:40:21,VHBEGHEWBEZNJ,F,34,195.162.9.84,1
23826,1746213,228134,2018-01-05 22:40:14,VHBEGHEWBEZNJ,F,34,195.162.9.84,0
26020,1746213,298353,2018-01-05 22:40:16,VHBEGHEWBEZNJ,F,34,195.162.9.84,1


,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
17073,1746213,370816,2018-01-01 11:32:42,UTIAECBUVOMYW,M,42,73.163.132.7,1
32426,1746213,235435,2018-01-01 11:32:49,UTIAECBUVOMYW,M,42,73.163.132.7,1
32606,1746213,191647,2018-01-01 11:32:33,UTIAECBUVOMYW,M,42,73.163.132.7,0
37867,1746213,352849,2018-01-01 11:32:41,UTIAECBUVOMYW,M,42,73.163.132.7,1
45148,1746213,198346,2018-01-01 11:32:44,UTIAECBUVOMYW,M,42,73.163.132.7,1
52821,1746213,392531,2018-01-01 11:32:40,UTIAECBUVOMYW,M,42,73.163.132.7,1


,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
8352,1746213,53272,2018-05-17 03:57:43,XHTRFYTLPDONI,M,27,36.103.7.66,1
10136,1746213,281586,2018-05-20 07:34:15,XHTRFYTLPDONI,M,36,87.118.127.218,0


The assumption seems mostly correct, but there may or may not be a 'victim', and just frauds with the same device.

Also we can see some patterns:
- E-commerse provider with id '1746213' shows up everywhere in the frauds registrations, 
- registration date differs within a few seconds,
- first registered user (time) is usually not marked as fraudster, that's probably most important info to distinguish
- registered user data is always the same.

Maybe it's worth checking if ip adresses are from the same country.

##### Processing feature table

By the way, I am also wondering how this information would fit into a regression model (for example):

In [227]:
hash('RJEAMRBBMPWEA')

-6868881713447379658

In [228]:
hash('JFVHSUGKDAYZV')

4746003862373081005

In [229]:
hashed_device_id = merchant_dataset_train['Registered_Device_ID'].apply(lambda x: hash(x))
hashed_device_id

0       -1852335909620913077
1       -6058622246119234625
2        4746003862373081005
3       -6871891618127599014
4        7943116118150859418
                ...         
54208    8266053671127450110
54209   -9045387780610934615
54210     819550870195332364
54211    -630140347673677412
54212   -4259516498185977922
Name: Registered_Device_ID, Length: 54213, dtype: int64

Seems hashing isnt that good idea... big negatives can be misleading (I think)

In [230]:
[ord(x) for x in 'LJCILLBRQZNKS']

[76, 74, 67, 73, 76, 76, 66, 82, 81, 90, 78, 75, 83]

But that seems alright, we can add category with ord(letter) and add them as features (1st letter, 2nd, 3rd, etc...)

##### TODO: ADD TO PIPE ML

In [231]:
devices = merchant_dataset_train['Registered_Device_ID']
device_id_splitted = devices.str.split('', expand=True, n=0)
device_id_splitted = device_id_splitted.drop(columns=[0, 14], axis = 1)
device_id_splitted = device_id_splitted.applymap(lambda x: ord(x))
device_id_splitted

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,86,65,84,81,77,77,90,84,86,79,90,85,84
1,76,74,67,73,76,76,66,82,81,90,78,75,83
2,74,70,86,72,83,85,71,75,68,65,89,90,86
3,87,70,82,88,77,80,76,81,89,88,82,77,89
4,71,71,72,75,87,77,83,87,72,67,77,73,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54208,75,71,86,77,69,78,71,67,73,65,86,79,75
54209,81,74,71,80,79,89,68,79,78,84,89,67,78
54210,90,79,74,78,65,74,77,71,78,82,81,80,90
54211,77,81,65,79,77,78,90,70,65,72,90,76,83


Now it's ready to join with the main dataset

#### Ecommerce_Provider_ID

In [232]:
merchant_all_data_train[merchant_all_data_train['Ecommerce_Provider_ID'] == 1746213]

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Fraudster
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,0
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,0
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,0
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,0
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,0
...,...,...,...,...,...,...,...,...
54208,1746213,222372,2018-05-01 21:58:19,KGVMENGCIAVOK,M,43,247.25.244.31,0
54209,1746213,73491,2018-06-11 19:54:13,QJGPOYDONTYCN,M,41,80.30.170.122,1
54210,1746213,176456,2018-01-06 20:05:55,ZOJNAJMGNRQPZ,F,36,188.39.154.217,0
54211,1746213,228277,2018-03-09 07:08:10,MQAOMNZFAHZLS,M,18,116.20.101.178,0


In [233]:
merchant_all_data_train['Ecommerce_Provider_ID'].value_counts()

1746213    54213
Name: Ecommerce_Provider_ID, dtype: int64

Wait... they are all registered under the same e-commerce provider... Should've drop that column, its useless.

##### TODO: ADD TO PIPE EDA

In [234]:
merchant_dataset_train.drop(columns=['Ecommerce_Provider_ID'])

,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,IP_converted_to_int,Country
0,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,815237196,NaN
1,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,1577685417,NaN
2,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,979279225,NaN
3,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,3250268602,NaN
4,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,2103448748,NaN
...,...,...,...,...,...,...,...,...
54208,222372,2018-05-01 21:58:19,KGVMENGCIAVOK,M,43,247.25.244.31,4145673247,NaN
54209,73491,2018-06-11 19:54:13,QJGPOYDONTYCN,M,41,80.30.170.122,1344187002,NaN
54210,176456,2018-01-06 20:05:55,ZOJNAJMGNRQPZ,F,36,188.39.154.217,3156712153,NaN
54211,228277,2018-03-09 07:08:10,MQAOMNZFAHZLS,M,18,116.20.101.178,1947493810,NaN


#### Date and time

About date-time: even if at first glance seconds of creation matter the most, after quite extensive research, it might be good idea to keep all numbers as separate features (years, mo, day, hour, minute, second)

https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96

##### Processing feature table

In [235]:
def convert_to_week(month, day):
    # Helper function to get the number of week in the year
    total_days = day

    for current_month in range(1, int(month)+1):
        if month == 2:
            total_days += 28
        elif month in [1, 3, 5, 7, 8, 10, 12]:
            total_days += 31
        else:
            total_days += 30

    return int(min(total_days // 7 + 1, 52 * 7))

##### TODO: ADD TO PIPE EDA

In [236]:
regex_datetime_split_pattern = '-| |:'
date_and_time_splitted = pd.DataFrame(columns=['Year', 'Week', 'Month', 'Day', 'Hour', 'Minute', 'Second'])
date_and_time_splitted[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = \
    merchant_dataset_train['Merchant_Registration_Date'].str.split(regex_datetime_split_pattern, n=6, expand=True)
date_and_time_splitted = date_and_time_splitted.applymap(int, na_action='ignore')
date_and_time_splitted['Week'] = date_and_time_splitted.apply(lambda x: convert_to_week(x.Month, x.Day), axis=1)
date_and_time_splitted

,Year,Week,Month,Day,Hour,Minute,Second
0,2018,23,5,1,21,15,11
1,2018,20,4,14,10,13,0
2,2018,29,6,20,7,44,22
3,2018,30,6,27,1,41,39
4,2018,6,1,5,0,55,41
...,...,...,...,...,...,...,...
54208,2018,23,5,1,21,58,19
54209,2018,28,6,11,19,54,13
54210,2018,6,1,6,20,5,55
54211,2018,15,3,9,7,8,10


In [237]:
date_and_time_splitted['Year'].value_counts()

2018    54213
Name: Year, dtype: int64

Since all are the same year, we can basically drop it.

##### TODO: ADD TO PIPE EDA

In [238]:
date_and_time_splitted = date_and_time_splitted.drop(columns='Year')

Now it's ready to join with the main dataset

#### Age and gender

In [239]:
merchant_dataset_train['Age'].value_counts()

32    2460
31    2399
29    2326
33    2316
28    2279
34    2279
30    2274
35    2242
36    2208
27    2090
37    2005
26    1984
38    1909
25    1868
24    1776
39    1734
23    1668
40    1628
41    1517
22    1376
42    1336
21    1329
20    1238
43    1177
19    1077
18    1004
44     970
45     903
46     818
47     717
48     631
49     517
50     436
51     313
52     309
53     246
54     215
55     150
56     122
57      95
58      86
59      49
60      35
61      34
63      20
62      18
64      11
65       7
66       4
67       3
68       2
69       2
72       1
Name: Age, dtype: int64

In [240]:
merchant_dataset_train['Age'].describe()

count    54213.000000
mean        33.122240
std          8.630091
min         18.000000
25%         27.000000
50%         32.000000
75%         39.000000
max         72.000000
Name: Age, dtype: float64

Age looks normal, somewhat unimpressive

In [241]:
gender = merchant_dataset_train['Gender']

In [242]:
gender.value_counts()

M    31761
F    22452
Name: Gender, dtype: int64

In [243]:
gender.describe()

count     54213
unique        2
top           M
freq      31761
Name: Gender, dtype: object

In [244]:
male_ratio = gender[gender == 'M'].count() / gender.count()
male_ratio

0.5858557910464279

In [245]:
female_ratio = gender[gender == 'F'].count() / gender.count()
female_ratio

0.414144208953572

No obvious patterns here too, I guess...

#### Visualization

Just for self-explanatory visual purposes, will not use the device_id, as we already somewhat know the pattern there, and enumerate the gender and countries, instead of one-hot-encoding.

##### TODO: ADD TO PIPE ML

In [246]:
vis_merchant_all_data_train = merchant_dataset_train.drop(
    columns=[
        'Ecommerce_Provider_ID', 
        'Merchant_Registration_Date', 
        'Registered_Device_ID', 
        'IP_Address', 
        'IP_converted_to_int'])

In [247]:
LE = LabelEncoder()
vis_merchant_all_data_train['Country'] = LE.fit_transform(vis_merchant_all_data_train['Country'])
vis_merchant_all_data_train['Gender'] = LE.fit_transform(vis_merchant_all_data_train['Gender'])

In [248]:
vis_merchant_all_data_train = vis_merchant_all_data_train.join([date_and_time_splitted])

In [249]:
vis_merchant_all_data_train = pd.merge(left=vis_merchant_all_data_train, right=target_dataset_train, on='Merchant_ID')

In [250]:
vis_merchant_all_data_train

,Merchant_ID,Gender,Age,Country,Week,Month,Day,Hour,Minute,Second,Fraudster
0,50448,0,39,0,23,5,1,21,15,11,0
1,338754,1,35,0,20,4,14,10,13,0,0
2,291127,0,40,0,29,6,20,7,44,22,0
3,319919,1,37,0,30,6,27,1,41,39,0
4,195911,0,27,0,6,1,5,0,55,41,0
...,...,...,...,...,...,...,...,...,...,...,...
54208,222372,1,43,0,23,5,1,21,58,19,0
54209,73491,1,41,0,28,6,11,19,54,13,1
54210,176456,0,36,0,6,1,6,20,5,55,0
54211,228277,1,18,0,15,3,9,7,8,10,0


##### !!! VIZ TURNED OFF

##### Visualisation thoughts

What we can see from the correlations is that fraudsters appear to be mostly registered in the first month(jan)/early weeks from the year and also in the first half of the month. 

Other than that, everything seems all over the place and regression models and SVM's would probably deliver poor results. 

What we know from the registered devices would exclude KNNs as we want exactly the opposite - sometimes to find the fraud between very similar datapoints.

So that basically leaves us to try DecisionTree/Forest or Naive Bayes, as for the latter I'm having doubts.

#### Additional preprocessing, cleanup and merge

##### TODO: ADD TO PIPE ML

Remove columns we wont need anymore and replace them with the preprocessed features. We can also One-Hot-Encode 'Country' and 'Gender' (perhaps gender converting to 0-1 is enough).

In [251]:
merchant_dataset_train = merchant_dataset_train.drop(columns=['Ecommerce_Provider_ID', 'Merchant_Registration_Date', 'Registered_Device_ID', 'IP_Address', 'IP_converted_to_int'])

In [252]:
LE = LabelEncoder()
merchant_dataset_train = pd.get_dummies(merchant_dataset_train, columns=['Country'])
merchant_dataset_train['Gender'] = LE.fit_transform(merchant_dataset_train['Gender'])

In [253]:
merchant_dataset_train = merchant_dataset_train.join([date_and_time_splitted, device_id_splitted])

In [254]:
merchant_dataset_train

,Merchant_ID,Gender,Age,Week,Month,Day,Hour,Minute,Second,1,...,4,5,6,7,8,9,10,11,12,13
0,50448,0,39,23,5,1,21,15,11,86,...,81,77,77,90,84,86,79,90,85,84
1,338754,1,35,20,4,14,10,13,0,76,...,73,76,76,66,82,81,90,78,75,83
2,291127,0,40,29,6,20,7,44,22,74,...,72,83,85,71,75,68,65,89,90,86
3,319919,1,37,30,6,27,1,41,39,87,...,88,77,80,76,81,89,88,82,77,89
4,195911,0,27,6,1,5,0,55,41,71,...,75,87,77,83,87,72,67,77,73,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54208,222372,1,43,23,5,1,21,58,19,75,...,77,69,78,71,67,73,65,86,79,75
54209,73491,1,41,28,6,11,19,54,13,81,...,80,79,89,68,79,78,84,89,67,78
54210,176456,0,36,6,1,6,20,5,55,90,...,78,65,74,77,71,78,82,81,80,90
54211,228277,1,18,15,3,9,7,8,10,77,...,79,77,78,90,70,65,72,90,76,83


### Orders Dataset

##### TODO: ADD TO PIPE EDA

In [255]:
order_all_data_train = pd.merge(left=order_dataset_train, right=target_dataset_train, on='Merchant_ID')

In [256]:
order_all_data_train

,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
0,126221,37cea9512f8d,2018-04-29 16:39:26,148,Direct,Credit Card,124231,0
1,115471,09f12e6efde2,2018-06-16 17:05:40,145,SEO,Credit Card,136178,0
2,151786,4e69e956e159,2018-10-26 18:00:46,62,Ads,Internet Banking,198611,1
3,140456,663443aaeb82,2018-12-12 05:41:52,28,SEO,Debit Card,127993,0
4,114721,99258810c121,2018-09-20 11:06:10,70,Ads,Credit Card,250146,0
...,...,...,...,...,...,...,...,...
54208,143943,4b0857877884,2018-06-01 14:39:47,90,Ads,Credit Card,61951,0
54209,125429,2de2749f5442,2018-08-05 21:58:02,120,SEO,Debit Card,328145,0
54210,159393,11635fac40ff,2018-06-23 01:54:24,92,Ads,Credit Card,28307,0
54211,133631,6bbcff2f14e5,2018-06-15 21:54:36,148,Ads,Cash On Delivery,277466,0


#### Customer ID
Shouldn't necessarily be unique, but lets check:

In [257]:
customer_orders = order_all_data_train['Customer_ID'].value_counts()
customer_orders

124702    7
154763    7
156576    7
111658    7
162123    6
         ..
151064    1
155976    1
127457    1
120799    1
133631    1
Name: Customer_ID, Length: 34081, dtype: int64

In [258]:
get_samples_from_counts(customer_orders, order_all_data_train, 'Customer_ID', 5)

,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
2274,124702,e76cc3815088,2018-11-06 08:20:25,78,SEO,Debit Card,249529,0
7635,124702,2b2953ea007b,2018-05-30 09:21:49,30,Ads,Internet Banking,177389,0
10346,124702,5b1d4ab9fd5f,2018-09-28 03:57:41,128,Ads,Internet Banking,352665,0
15453,124702,db75c4056958,2018-04-20 20:54:02,48,Ads,Debit Card,147814,0
34172,124702,1cd20e19ad31,2018-09-13 07:44:43,132,Direct,E-wallet,147244,0
41192,124702,4b97c53656b1,2018-07-26 06:28:28,82,Ads,E-wallet,399924,0
43387,124702,d4dcaee17fd4,2018-02-16 12:23:08,118,SEO,Debit Card,348063,0


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
25371,136752,00b510c4d4c9,2018-08-30 02:21:07,50,SEO,Internet Banking,19341,0
33856,136752,43643390d23c,2018-04-09 06:33:38,45,Ads,E-wallet,47293,0


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
21607,162581,3b6908c9fa01,2018-08-03 15:33:40,45,Direct,Internet Banking,177421,0
25308,162581,264cf83cee18,2018-01-04 17:48:08,35,SEO,Internet Banking,143720,1
25411,162581,d2e2fd95471f,2018-08-21 15:47:05,65,Ads,Credit Card,202427,0
37673,162581,501fc38eb729,2018-06-14 17:00:59,65,Ads,Debit Card,33184,0


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
5399,138899,cf014fbd8faa,2018-06-14 09:30:18,170,SEO,Credit Card,82516,0
14035,138899,a63fce8bc616,2018-03-31 22:15:54,75,Direct,E-wallet,394822,0
15975,138899,28578a9dead5,2018-09-01 12:37:31,70,Ads,Internet Banking,203138,0
19543,138899,57d3efc16b8a,2018-04-15 04:46:18,65,SEO,Credit Card,248116,0
43414,138899,9bbb8fd6af54,2018-07-23 20:48:42,150,SEO,E-wallet,357369,0


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
3348,161878,285613e2a500,2018-05-09 14:34:51,122,Direct,Cash On Delivery,66320,0
5935,161878,24f44ba0558f,2018-04-26 21:27:47,115,SEO,Internet Banking,275751,0
7908,161878,ea06f53de22f,2018-02-14 22:00:45,30,SEO,Debit Card,163887,0
8378,161878,ffd810031fb1,2018-05-04 17:45:05,125,SEO,Credit Card,309070,0
40624,161878,5d28e5f97670,2018-06-15 12:47:02,128,Ads,Internet Banking,109912,0
46086,161878,c7891c8219d6,2018-08-21 00:54:06,128,Ads,E-wallet,123254,0


In [259]:
victims = order_all_data_train['Customer_ID'][order_all_data_train['Fraudster'] == 1].value_counts()
victims

137312    3
143946    3
129340    3
146948    3
113936    3
         ..
142980    1
132526    1
116687    1
161358    1
160661    1
Name: Customer_ID, Length: 4803, dtype: int64

In [260]:
get_samples_from_counts(victims, order_all_data_train, 'Customer_ID', 5)

,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
3288,115277,0236c29f1327,2018-06-05 05:04:17,62,Direct,Credit Card,198484,0
12139,115277,792df06b2e3f,2018-01-12 02:11:35,95,SEO,Credit Card,270688,1
28271,115277,fd8b5ce5020a,2018-05-27 00:42:36,98,Ads,Credit Card,35247,1


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
1941,144042,a3dee38f5c11,2018-01-10 08:52:55,122,SEO,Internet Banking,139093,1
17322,144042,d31971d7412d,2018-02-04 05:43:26,25,SEO,Debit Card,376435,0
42939,144042,41a040e89759,2018-01-10 09:52:09,42,Ads,Credit Card,106367,1


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
20464,125187,b043602b709b,2018-07-06 22:15:05,35,Ads,Credit Card,113437,1
21062,125187,1fa5a1c2382a,2018-06-29 13:21:12,108,Direct,E-wallet,281780,1
34904,125187,0b26fe49a8c0,2018-01-14 01:04:41,125,Ads,Internet Banking,233822,1


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
15828,148560,c7c9635c7be7,2018-01-12 16:50:25,48,Ads,Credit Card,37683,1
27619,148560,145249b739c9,2018-07-23 00:55:52,140,SEO,Credit Card,85907,0
28557,148560,4cbeb28900f3,2018-04-12 21:07:12,42,Direct,Credit Card,295531,0
32656,148560,f9646f6a3099,2018-07-31 07:59:24,100,Ads,Internet Banking,141503,1
40219,148560,83abf61a98a4,2018-05-23 01:01:40,118,SEO,Internet Banking,277691,0
48242,148560,0ee00585b3e0,2018-09-13 13:09:43,60,SEO,Internet Banking,6238,0


,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
2321,129340,85137148337e,2018-01-10 06:55:23,32,SEO,Debit Card,62445,1
25660,129340,29bae5def3f9,2018-01-10 03:19:13,125,SEO,E-wallet,59904,1
44896,129340,d92edefa8615,2018-10-31 04:06:22,120,SEO,E-wallet,124822,1


Okay, so basically we have 1 order from each merchant, some of the customers may have a few orders and some of them have purchased from a fraudster.

What's more is that there is a pattern where some customers are more prone to a fraud and usually frauded more than once, but there are some 'occasional' ones as well.

#### Order ID #TODO hex to dec?
The data seems to be a unique hexadecimal number. We can convert it to decimals I guess... But do we need it at all? There is unique value for each record and it doesnt give us any insight... We could drop it anyway

In [261]:
order_all_data_train['Order_ID'].describe()

count            54213
unique           54213
top       37cea9512f8d
freq                 1
Name: Order_ID, dtype: object

In [262]:
order_all_data_train['Order_ID'] = order_all_data_train['Order_ID'].apply(lambda x: hex(int(x, 16)))

In [263]:
order_all_data_train

,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster
0,126221,0x37cea9512f8d,2018-04-29 16:39:26,148,Direct,Credit Card,124231,0
1,115471,0x9f12e6efde2,2018-06-16 17:05:40,145,SEO,Credit Card,136178,0
2,151786,0x4e69e956e159,2018-10-26 18:00:46,62,Ads,Internet Banking,198611,1
3,140456,0x663443aaeb82,2018-12-12 05:41:52,28,SEO,Debit Card,127993,0
4,114721,0x99258810c121,2018-09-20 11:06:10,70,Ads,Credit Card,250146,0
...,...,...,...,...,...,...,...,...
54208,143943,0x4b0857877884,2018-06-01 14:39:47,90,Ads,Credit Card,61951,0
54209,125429,0x2de2749f5442,2018-08-05 21:58:02,120,SEO,Debit Card,328145,0
54210,159393,0x11635fac40ff,2018-06-23 01:54:24,92,Ads,Credit Card,28307,0
54211,133631,0x6bbcff2f14e5,2018-06-15 21:54:36,148,Ads,Cash On Delivery,277466,0


#### Order date

No obvious patterns here. We can convert datetime to separate columns here as well.

##### TODO: ADD TO PIPE EDA

##### Processing feature table

In [264]:
regex_datetime_split_pattern = '-| |:'
date_of_order_splitted = pd.DataFrame(columns=['Year', 'Week', 'Month', 'Day', 'Hour', 'Minute', 'Second'])
date_of_order_splitted[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = \
    order_all_data_train['Date_of_Order'].str.split(regex_datetime_split_pattern, n=6, expand=True)
date_of_order_splitted = date_of_order_splitted.applymap(int, na_action='ignore')
date_of_order_splitted['Week'] = date_of_order_splitted.apply(lambda x: convert_to_week(x.Month, x.Day), axis=1)

In [265]:
date_of_order_splitted

,Year,Week,Month,Day,Hour,Minute,Second
0,2018,22,4,29,16,39,26
1,2018,29,6,16,17,5,40
2,2018,49,10,26,18,0,46
3,2018,55,12,12,5,41,52
4,2018,42,9,20,11,6,10
...,...,...,...,...,...,...,...
54208,2018,26,6,1,14,39,47
54209,2018,37,8,5,21,58,2
54210,2018,30,6,23,1,54,24
54211,2018,28,6,15,21,54,36


In [266]:
date_of_order_splitted['Week'].value_counts()

40    2834
27    2689
39    1898
18    1802
26    1787
24    1630
35    1619
34    1616
29    1591
6     1590
36    1589
37    1563
25    1548
28    1540
33    1539
30    1536
38    1533
21    1481
20    1421
32    1376
19    1284
41    1218
42    1191
48    1148
49    1108
23    1101
43    1067
17    1061
7     1048
16     954
46     944
22     903
15     843
47     843
9      646
12     643
11     605
10     522
45     521
14     426
50     425
51     340
8      236
31     232
5      215
52     142
54     122
13     116
55     107
56      20
Name: Week, dtype: int64

In [267]:
date_of_order_splitted['Year'].value_counts()

2018    54213
Name: Year, dtype: int64

Since all are the same year, we can drop it.

##### TODO: ADD TO PIPE EDA

In [268]:
date_of_order_splitted = date_of_order_splitted.drop(columns='Year')

In [269]:
date_of_order_splitted

,Week,Month,Day,Hour,Minute,Second
0,22,4,29,16,39,26
1,29,6,16,17,5,40
2,49,10,26,18,0,46
3,55,12,12,5,41,52
4,42,9,20,11,6,10
...,...,...,...,...,...,...
54208,26,6,1,14,39,47
54209,37,8,5,21,58,2
54210,30,6,23,1,54,24
54211,28,6,15,21,54,36


##### TODO: ADD TO PIPE EDA

In [270]:
order_all_data_train = order_all_data_train.drop(columns=['Date_of_Order'])

In [271]:
order_all_data_train = order_all_data_train.join(date_of_order_splitted)

#### Order value

Value of orders with no frauds

In [272]:
order_all_data_train['Order_Value_USD'][order_all_data_train['Fraudster'] == 0].describe()

count    49188.000000
mean        92.213934
std         45.635488
min         22.000000
25%         55.000000
50%         88.000000
75%        122.000000
max        385.000000
Name: Order_Value_USD, dtype: float64

---

Value of orders with frauds

In [273]:
order_all_data_train['Order_Value_USD'][order_all_data_train['Fraudster'] == 1].describe()

count    5025.000000
mean       92.389851
std        46.045665
min        22.000000
25%        55.000000
50%        88.000000
75%       122.000000
max       270.000000
Name: Order_Value_USD, dtype: float64

Can't see any patterns here as well... There is almost no difference between value of orders with frauds and without.

#### Order Source

In [274]:
order_all_data_train['Order_Source'][order_all_data_train['Fraudster'] == 0].value_counts()

SEO       19991
Ads       19298
Direct     9899
Name: Order_Source, dtype: int64

In [275]:
order_all_data_train['Order_Source'][order_all_data_train['Fraudster'] == 1].value_counts()

Ads       1961
SEO       1893
Direct    1171
Name: Order_Source, dtype: int64

Looks like direct orders are more prone to frauds.

#### Order payment method

In [276]:
order_all_data_train['Order_Payment_Method'][order_all_data_train['Fraudster'] == 0].value_counts()

Credit Card         19682
Internet Banking    12059
Debit Card           8254
E-wallet             7938
Cash On Delivery     1255
Name: Order_Payment_Method, dtype: int64

In [277]:
order_all_data_train['Order_Payment_Method'][order_all_data_train['Fraudster'] == 1].value_counts()

Credit Card         2162
Internet Banking    1134
Debit Card           816
E-wallet             801
Cash On Delivery     112
Name: Order_Payment_Method, dtype: int64

Can't see any patterns here as well... There is almost no difference between payment methods with frauds and without.

#### Visualization

##### TODO: ADD TO PIPE ML

In [278]:
vis_order_all_data_train = order_all_data_train.copy()

In [279]:
LE = LabelEncoder()
vis_order_all_data_train['Order_Source'] = LE.fit_transform(vis_order_all_data_train['Order_Source'])
vis_order_all_data_train['Order_Payment_Method'] = LE.fit_transform(vis_order_all_data_train['Order_Payment_Method'])

In [280]:
vis_order_all_data_train

,Customer_ID,Order_ID,Order_Value_USD,Order_Source,Order_Payment_Method,Merchant_ID,Fraudster,Week,Month,Day,Hour,Minute,Second
0,126221,0x37cea9512f8d,148,1,1,124231,0,22,4,29,16,39,26
1,115471,0x9f12e6efde2,145,2,1,136178,0,29,6,16,17,5,40
2,151786,0x4e69e956e159,62,0,4,198611,1,49,10,26,18,0,46
3,140456,0x663443aaeb82,28,2,2,127993,0,55,12,12,5,41,52
4,114721,0x99258810c121,70,0,1,250146,0,42,9,20,11,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54208,143943,0x4b0857877884,90,0,1,61951,0,26,6,1,14,39,47
54209,125429,0x2de2749f5442,120,2,2,328145,0,37,8,5,21,58,2
54210,159393,0x11635fac40ff,92,0,1,28307,0,30,6,23,1,54,24
54211,133631,0x6bbcff2f14e5,148,0,0,277466,0,28,6,15,21,54,36


##### !!! VIZ TURNED OFF

All over the place again. Same patterns with the month and day (Mostly January, and the first half of the months). Perhaps we can use number of week in the year instead of month/day.

The observation that some of the victims are actually more prone, lets call them 'serial victims', kind of appears again (Customer_ID > 140,000) which are maybe new(unexperienced) or younger customers.

#### Merchant ID

In [281]:
order_dataset_train['Merchant_ID'].value_counts()

124231    1
11699     1
373439    1
148049    1
96865     1
         ..
83413     1
329499    1
159879    1
165373    1
157675    1
Name: Merchant_ID, Length: 54213, dtype: int64

In [282]:
order_dataset_merged = pd.concat([order_dataset_train, order_dataset_test])
order_dataset_merged['Merchant_ID'].value_counts()

124231    1
304866    1
356541    1
136474    1
3312      1
         ..
22733     1
251047    1
368945    1
12487     1
319050    1
Name: Merchant_ID, Length: 67767, dtype: int64

Hmm... only 1 order per merchant? Seems phishy... and unfortunate for our future model...

#### Additional preprocessing

##### TODO: ADD TO PIPE ML

In [283]:
order_dataset_train = order_all_data_train.drop(columns=['Fraudster'])

In [284]:
order_dataset_train = pd.get_dummies(order_dataset_train, columns=['Order_Source', 'Order_Payment_Method'])

#### Datasets Preprocessing pipeline

##### TODO

## ML algorithm

Ok, so, after all, we have the following datasets, which should be ready for our model:

In [289]:
merchant_dataset_train

,Merchant_ID,Gender,Age,Week,Month,Day,Hour,Minute,Second,1,...,5,6,7,8,9,10,11,12,13,Country
0,50448,0,39,23,5,1,21,15,11,86,...,77,77,90,84,86,79,90,85,84,NaN
1,338754,1,35,20,4,14,10,13,0,76,...,76,76,66,82,81,90,78,75,83,NaN
2,291127,0,40,29,6,20,7,44,22,74,...,83,85,71,75,68,65,89,90,86,NaN
3,319919,1,37,30,6,27,1,41,39,87,...,77,80,76,81,89,88,82,77,89,NaN
4,195911,0,27,6,1,5,0,55,41,71,...,87,77,83,87,72,67,77,73,68,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54208,222372,1,43,23,5,1,21,58,19,75,...,69,78,71,67,73,65,86,79,75,NaN
54209,73491,1,41,28,6,11,19,54,13,81,...,79,89,68,79,78,84,89,67,78,NaN
54210,176456,0,36,6,1,6,20,5,55,90,...,65,74,77,71,78,82,81,80,90,NaN
54211,228277,1,18,15,3,9,7,8,10,77,...,77,78,90,70,65,72,90,76,83,NaN


In [291]:
order_dataset_train

,Customer_ID,Order_ID,Order_Value_USD,Merchant_ID,Week,Month,Day,Hour,Minute,Second,Order_Source_Ads,Order_Source_Direct,Order_Source_SEO,Order_Payment_Method_Cash On Delivery,Order_Payment_Method_Credit Card,Order_Payment_Method_Debit Card,Order_Payment_Method_E-wallet,Order_Payment_Method_Internet Banking
0,126221,0x37cea9512f8d,148,124231,22,4,29,16,39,26,0,1,0,0,1,0,0,0
1,115471,0x9f12e6efde2,145,136178,29,6,16,17,5,40,0,0,1,0,1,0,0,0
2,151786,0x4e69e956e159,62,198611,49,10,26,18,0,46,1,0,0,0,0,0,0,1
3,140456,0x663443aaeb82,28,127993,55,12,12,5,41,52,0,0,1,0,0,1,0,0
4,114721,0x99258810c121,70,250146,42,9,20,11,6,10,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54208,143943,0x4b0857877884,90,61951,26,6,1,14,39,47,1,0,0,0,1,0,0,0
54209,125429,0x2de2749f5442,120,328145,37,8,5,21,58,2,0,0,1,0,0,1,0,0
54210,159393,0x11635fac40ff,92,28307,30,6,23,1,54,24,1,0,0,0,1,0,0,0
54211,133631,0x6bbcff2f14e5,148,277466,28,6,15,21,54,36,1,0,0,1,0,0,0,0
